- a set of auxiliary circuits are probabilistically sampled -> executed on a noisy backend -> post processing of noisy result to get error mitigated result -> probabilistically remove the noise of a quantum computer
- represent each ideal unitary gate G in a circuit as an average over a set of noisy gates {Oα},weighted by a real quasi-probability distribution η(α)
  G = ∑η(α)Oα = sum over noisy operations Oα
  ∑ η(α) = 1 (trace-preserving condition) G and {Oα} linear super-operators acting on density matrices.
-  ideal expectation value can be estimated as a Monte Carlo average over different noisy circuits,sampled according to the quasi-probability distributions associated to the ideal gates
- due to negativity of η(α) the required number of Monte Carlo samples can be large.
- knowing perfect tomography of Oα perfectly cancel hardware noise.
- NoisyOperation(circuit, channel_matrix=None) : operation (or sequence of operations)which a noisy quantum computer can actually implement.
- define noisy operations{Oα} (by NoisyOperation) -> associate to each operation the corresponding quasi-probability η(α) -> OperationRepresentation  -> apply PEC via the function execute_with_pec
  OperationRepresentation(ideal, noisy_operations, coeffs, is_qubit_dependent=True): basis expansion of an operation or sequence of operations in a basis of noisy, implementable operations.
  
- determine the quasi-probability representations:
  1.hardware noise model is well approximated by a simplified theoretical quantum channel(e.g. depolarizing or amplitude damping) -> apply known analytical expressions.
  2.complete process tomography of a basis set of implementable noisy operations (e.g. the native gate set of the backend)
  find_optimal_representation():
 - execute_with_pec : internally performs the Monte Carlo sampling process.
   -noisy_operation , sign , eta = operation_rep . sample () :returns the associated coefficient (use for manual sampling)
   
   -sample_circuit() :sampling an entire implementable circuit from the quasiprobability representation of an ideal circuit
 
 
 
 -Disadvantages:
. number of samples grows exponentially with respect to the circuit size and to the amount of noise.
.for ideal expectation value,quasi-probability representations of all the gates are known with sufficiently large accuracy.
.full tomography of the noisy gates is typically
necessary in order to build the quasi-probability representations for the ideal gates.

 
 
 
  
  

In [1]:
import mitiq
mitiq.SUPPORTED_PROGRAM_TYPES.keys()

dict_keys(['cirq', 'pyquil', 'qiskit', 'braket', 'pennylane', 'qibo'])

In [2]:
frontend = "qiskit"

In [3]:
# define the circuit of interest
from mitiq import benchmarks
circuit = benchmarks.generate_rb_circuits(n_qubits=1, num_cliffords=2, return_type=frontend,)[0] #idealy identity
# rb_circuits :randomized-benchmarking circuit 
#  num_cliffords is the number of random Clifford group elements and is proportional to the depth of the circuit.

print(circuit)

   ┌──────┐┌─────────┐┌─────────┐┌───────┐┌───┐┌────┐
q: ┤ √Xdg ├┤ Ry(π/2) ├┤ Ry(π/2) ├┤ Rx(0) ├┤ Y ├┤ √X ├
   └──────┘└─────────┘└─────────┘└───────┘└───┘└────┘


In [4]:
# define an executor which simulates a backend with depolarizing noise and computes the survival probability of the 00 state.
# we first convert the input circuit to the Mitiq internal representation (Cirq) and then simulate it.
from cirq import DensityMatrixSimulator, depolarize
from mitiq.interface import convert_to_mitiq

def execute(circuit, noise_level=0.01):
    """Returns Tr[ρ |0⟩⟨0|] where ρ is the state prepared by the circuit
    executed with depolarizing noise.
    """
    # Replace with code based on your frontend and backend.
    mitiq_circuit, _  = convert_to_mitiq(circuit) #  what is means(  , _  ) 
    # We add a simple noise model to simulate a noisy backend.
    noisy_circuit = mitiq_circuit.with_noise(depolarize(p=noise_level))
    rho = DensityMatrixSimulator().simulate(noisy_circuit).final_density_matrix
    return rho[0, 0].real

In [5]:
# function execute can be used to evaluate noisy (unmitigated) expectation values.
# Compute the expectation value of the |0⟩⟨0| observable.
noisy_value = execute(circuit)
ideal_value = execute(circuit, noise_level=0.0)  # noise_level=0.0 is it overwrite the defined value?
print(f"Error without mitigation: {abs(ideal_value - noisy_value) :.5f}") # how it is written 

Error without mitigation: 0.03869


Applying PEC:
-represent some or all of ideal gates as a linearcombinations of noisy gates(quasi-probability representations).In Mitiq, they correspond to OperationRepresentation objects.

How can we obtain the quasi-probability representations that are appropriate for a given backend? 


    Case 1: The noise of the backend can be approximated by a simple noise model, such that quasi-probability representations can be analytically computed. For example, this is possible for depolarizing or amplitude damping noise.

    Case 2: The noise of the backend is too complex and cannot be approximated by a simple noise model.



Method for case 1: A simple noise model (e.g. depolarizing or amplitude damping) is typically characterized by a single noise_level parameter (or a few parameters) that can be experimentally estimated. Possible techniques for estimating the noise level are randomized-benchmarking experiments or calibration experiments. Often, gate error probabilities are reported by hardware vendors and can be used to obtain a good guess for the noise_level without running any experiments. Given the noise model and the noise_level, one can apply known analytical expressions to compute the quasi-probability representations of arbitrary gates.

Method for case 2: Assuming an over-simplified noise model may be a bad approximation. In this case, the suggested approach is to perform the complete process tomography of a basis set of implementable noisy operations (e.g. the native gate set of the backend). One could also use gate set tomography (GST), a noise characterization technique which is robust to state-preparation and measurement errors. Given the superoperators of the noisy implementable operations, one can obtain the quasi-probability representations as solutions of numerical optimization problems. In Mitiq, this is possible through the find_optimal_representation() function.




In [6]:
# the execute function simulates the effect of depolarizing noise of some fixed noise_level acting after each gate
from mitiq.pec.representations.depolarizing import represent_operations_in_circuit_with_local_depolarizing_noise

noise_level = 0.01 #same as defined in execute() function
reps = represent_operations_in_circuit_with_local_depolarizing_noise(circuit, noise_level)
print(f"{len(reps)} OperationRepresentation objects produced, assuming {100 * noise_level}% depolarizing noise.")

5 OperationRepresentation objects produced, assuming 1.0% depolarizing noise.


In [7]:
# print the first OperationRepresentation in reps, showing how an ideal gate can be expressed as a linear combination of noisy gates:
print(reps[0])

q_0: ───Ry(0.5π)─── = 1.010*(q_0: ───Ry(0.5π)───)-0.003*(q_0: ───Ry(0.5π)───X───)-0.003*(q_0: ───Ry(0.5π)───Y───)-0.003*(q_0: ───Ry(0.5π)───Z───)


In [8]:
# For PEC to work properly, the noise model and the noise_level associated to the execute function must correspond to those used to compute the OperationRepresentation objects.

# Once the necessary OperationRepresentations have been defined, probabilistic error cancellation can be applied through the execute_with_pec() function as follows:
from mitiq import pec

pec_value = pec.execute_with_pec(circuit, execute, representations=reps)

print(f"Error without PEC: {abs(ideal_value - noisy_value) :.5f}")
print(f"Error with PEC:    {abs(ideal_value - pec_value) :.5f}")

Error without PEC: 0.03869
Error with PEC:    0.01128


In [1]:
import cirq
from mitiq import zne, benchmarks


def execute(circuit, noise_level=0.005):
    """Returns Tr[ρ |0⟩⟨0|] where ρ is the state prepared by the circuit
    with depolarizing noise."""
    noisy_circuit = circuit.with_noise(cirq.depolarize(p=noise_level))
    return (
        cirq.DensityMatrixSimulator()
        .simulate(noisy_circuit)
        .final_density_matrix[0, 0]
        .real
    )


circuit = benchmarks.generate_rb_circuits(n_qubits=1, num_cliffords=50)[0]

true_value = execute(circuit, noise_level=0.0)      # Ideal quantum computer
noisy_value = execute(circuit)                      # Noisy quantum computer
zne_value = zne.execute_with_zne(circuit, execute)  # Noisy quantum computer + Mitiq

print(f"Error w/o  Mitiq: {abs((true_value - noisy_value) / true_value):.3f}")
print(f"Error w Mitiq:    {abs((true_value - zne_value) / true_value):.3f}")

Error w/o  Mitiq: 0.260
Error w Mitiq:    0.071


In [12]:
import numpy as np
import qiskit # Frontend library
from mitiq import pec  # Probabilistic error cancellation module

In [13]:
basis_circuit_h = qiskit.QuantumCircuit(1)
basis_circuit_h.h(0)

basis_circuit_hx = qiskit.QuantumCircuit(1)
basis_circuit_hx.h(0)
basis_circuit_hx.x(0)

basis_circuit_hy = qiskit.QuantumCircuit(1)
basis_circuit_hy.h(0)
basis_circuit_hy.y(0)

basis_circuit_hz = qiskit.QuantumCircuit(1)
basis_circuit_hz.h(0)
basis_circuit_hz.z(0)

basis_circuits = [basis_circuit_h, basis_circuit_hx, basis_circuit_hy, basis_circuit_hz] 

for c in basis_circuits:
    print(c)

   ┌───┐
q: ┤ H ├
   └───┘
   ┌───┐┌───┐
q: ┤ H ├┤ X ├
   └───┘└───┘
   ┌───┐┌───┐
q: ┤ H ├┤ Y ├
   └───┘└───┘
   ┌───┐┌───┐
q: ┤ H ├┤ Z ├
   └───┘└───┘


In [14]:
from mitiq.pec.representations import local_depolarizing_kraus
from mitiq.pec.channels import kraus_to_super

# Compute depolarizing superoperator
BASE_NOISE = 0.2
depo_super = kraus_to_super(local_depolarizing_kraus(BASE_NOISE, num_qubits=1))

# Define the superoperator matrix of each noisy operation
super_matrices = [
    depo_super @ kraus_to_super([qiskit.quantum_info.Operator(c).data]) 
    for c in basis_circuits
]

# Define NoisyOperation objects combining circuits with channel matrices
noisy_operations = [
    pec.NoisyOperation(circuit=c, channel_matrix=m)
    for c, m in zip(basis_circuits, super_matrices)
]

print(f"{len(noisy_operations)} NoisyOperation objects defined.")

4 NoisyOperation objects defined.


In [15]:
ideal_operation = qiskit.QuantumCircuit(1)
ideal_operation.h(0)
print(f"The ideal operation to expand in the noisy basis is:\n{ideal_operation}")

The ideal operation to expand in the noisy basis is:
   ┌───┐
q: ┤ H ├
   └───┘


In [16]:
from mitiq.pec.representations import find_optimal_representation

h_rep = find_optimal_representation(ideal_operation, noisy_operations)
print(f"Optimal representation:\n{h_rep}")

Optimal representation:
q_0: ───H─── = 1.273*(   ┌───┐
q: ┤ H ├
   └───┘)-0.091*(   ┌───┐┌───┐
q: ┤ H ├┤ X ├
   └───┘└───┘)-0.091*(   ┌───┐┌───┐
q: ┤ H ├┤ Y ├
   └───┘└───┘)-0.091*(   ┌───┐┌───┐
q: ┤ H ├┤ Z ├
   └───┘└───┘)


In [17]:
# We assume to know the quasi-distribution
quasi_dist = h_rep.coeffs

# Manual definition of the OperationRepresentation
manual_h_rep = pec.OperationRepresentation(
    ideal=ideal_operation,
    noisy_operations=noisy_operations,
    coeffs=quasi_dist,
)

# Test that the manual definition is equivalent to h_rep
assert manual_h_rep == h_rep

In [18]:
from mitiq.pec.representations.depolarizing import represent_operation_with_local_depolarizing_noise

depolarizing_h_rep = represent_operation_with_local_depolarizing_noise(
    ideal_operation,
    noise_level=BASE_NOISE,
)

assert depolarizing_h_rep == h_rep

In [19]:
qreg = qiskit.QuantumRegister(2) 
circuit = qiskit.QuantumCircuit(qreg)
circuit.h(0)
circuit.h(1)

# OperationRepresentation defined on different qubits
rep_qreg = qiskit.QuantumRegister(1, "rep_reg") 
ideal_op = qiskit.QuantumCircuit(rep_qreg)
ideal_op.h(rep_qreg)
hxcircuit = qiskit.QuantumCircuit(rep_qreg)
hxcircuit.h(0)
hxcircuit.x(0)
hzcircuit = qiskit.QuantumCircuit(rep_qreg)
hzcircuit.h(0)
hzcircuit.z(0)
noisy_hxop = pec.NoisyOperation(hxcircuit)
noisy_hzop = pec.NoisyOperation(hzcircuit)

rep = pec.OperationRepresentation(
    ideal=ideal_op,
    noisy_operations=[noisy_hxop, noisy_hzop],
    coeffs=[0.5, 0.5],
    is_qubit_dependent=False,
)
print(rep)
print("Using the same rep on a circuit with H gates acting on different qubits:")
sampled_circuits, _, _ = pec.sample_circuit(circuit, representations=[rep])
print(*sampled_circuits)

rep_reg_0: ───H─── = 0.500*(         ┌───┐┌───┐
rep_reg: ┤ H ├┤ X ├
         └───┘└───┘)+0.500*(         ┌───┐┌───┐
rep_reg: ┤ H ├┤ Z ├
         └───┘└───┘)
Using the same rep on a circuit with H gates acting on different qubits:
      ┌───┐┌───┐
q1_0: ┤ H ├┤ X ├
      ├───┤├───┤
q1_1: ┤ H ├┤ Z ├
      └───┘└───┘


In [20]:
noisy_op, sign, coeff = h_rep.sample()
print(f"The sampled noisy operation is: {noisy_op}.")
print(f"The associated coefficient is {coeff:g}, whose sign is {sign}.")

The sampled noisy operation is:    ┌───┐
q: ┤ H ├
   └───┘.
The associated coefficient is 1.27273, whose sign is 1.
